In [3]:
import numpy as np
import xarray as xr
import numpy as np
from joblib import load, dump
import json

### Function to divide a dataset into individual datasets for each campground with the coordinates stored in the "campgrounds_coordinates.pkl" file. The window size is 35x35km around the campground. The final file is a dictionary, from which the individual datasets can be accessed like, for example, output_data['Bern'].

In [2]:
def camp_ds(ds_agg):

    camp_coord = load('campgrounds_coordinates.pkl')

    half_window = 35 // 2

    output_data = {}

    for _, row in camp_coord.iterrows():
        name = row["name"]
        x_center = row["x_coordinate"]
        y_center = row["y_coordinate"]

        # Find nearest indices in x and y
        x_idx = np.argmin(np.abs(ds_agg.x.values - x_center))
        y_idx = np.argmin(np.abs(ds_agg.y.values - y_center))

        # Define slices (handle edge cases if necessary)
        x_slice = slice(x_idx - half_window, x_idx + half_window + 1)
        y_slice = slice(y_idx - half_window, y_idx + half_window + 1)

        # Extract window
        data_window = ds_agg.isel(x=x_slice, y=y_slice)

        # Store in dictionary
        output_data[name] = data_window

    return output_data

### Open a dataset of aggregated forecasts and apply the function above

In [3]:
ds = xr.open_dataset('/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_07_2024_agg.nc')
ds_camp = camp_ds(ds)

### Open the json file with the precipitation thresholds of the individual campgrounds

In [5]:
with open("warnlevels.json", "r") as f:
    thresholds_json = json.load(f)

### This code scans the above opened dataset for cases (meaning dates and corresponding lead times) when a warning would have been issued

In [6]:
results = {}  # to store results per location

for location, ds in ds_camp.items():
    thresholds = thresholds_json[location]  # your JSON dict for this location
    dates_times = []

    # Iterate over date/time combinations
    for date_idx, date_val in enumerate(ds.date.values):
        for time_idx, time_val in enumerate(ds.time.values):

            warn_levels = np.zeros(ds.members.size, dtype=int)  # default 0

            # Check each member individually
            for member_idx in range(ds.members.size):
                precip = ds.tot_prec_1h.isel(
                    date=date_idx, time=time_idx, members=member_idx
                ).values  # shape (y, x)

                # Go from highest to lowest warn level
                for level_str in sorted(thresholds.keys(), key=int, reverse=True):
                    precip_thresh, area_thresh = thresholds[level_str]

                    frac_area = (precip >= precip_thresh).mean()
                    if frac_area >= area_thresh:
                        warn_levels[member_idx] = int(level_str)
                        break  # stop checking lower levels

            # Count members with warn level 2 or 3
            count_high = np.sum(np.isin(warn_levels, [3]))
            if count_high >= 8:
                # Convert date_val to numpy datetime64 before formatting
                dates_times.append((date_val, time_val))

    results[location] = dates_times


### Now you can save the results as a .pkl file for all the available summer months. Keep in mind to just change the month of the file that is read in above.

In [ ]:
#dump(results,'september_warn_times.pkl')

In [6]:
july_warn_times = load('july_warn_times.pkl')

In [18]:
july_warn_times['Lugano']

[(np.str_('2024071118'), np.int64(12)), (np.str_('2024071121'), np.int64(12))]

In [12]:
july_warn_times['Gordevio']

[(np.str_('2024071103'), np.int64(24)),
 (np.str_('2024071112'), np.int64(18)),
 (np.str_('2024071118'), np.int64(12)),
 (np.str_('2024071200'), np.int64(0))]

In [7]:
august_warn_times = load('august_warn_times.pkl')

In [20]:
august_warn_times['Interlaken']

[(np.str_('2024081109'), np.int64(24)),
 (np.str_('2024081112'), np.int64(24)),
 (np.str_('2024081115'), np.int64(18)),
 (np.str_('2024081118'), np.int64(18)),
 (np.str_('2024081121'), np.int64(12)),
 (np.str_('2024081203'), np.int64(6)),
 (np.str_('2024081209'), np.int64(0)),
 (np.str_('2024081212'), np.int64(0)),
 (np.str_('2024081309'), np.int64(24)),
 (np.str_('2024081315'), np.int64(0)),
 (np.str_('2024081509'), np.int64(24)),
 (np.str_('2024081512'), np.int64(24)),
 (np.str_('2024081521'), np.int64(18)),
 (np.str_('2024081600'), np.int64(12)),
 (np.str_('2024081603'), np.int64(6)),
 (np.str_('2024081603'), np.int64(12)),
 (np.str_('2024081606'), np.int64(6)),
 (np.str_('2024081609'), np.int64(6)),
 (np.str_('2024081612'), np.int64(0)),
 (np.str_('2024081615'), np.int64(0)),
 (np.str_('2024082821'), np.int64(12)),
 (np.str_('2024082903'), np.int64(6)),
 (np.str_('2024083012'), np.int64(24)),
 (np.str_('2024083100'), np.int64(12)),
 (np.str_('2024083109'), np.int64(24)),
 (np.str_(

In [13]:
august_warn_times['Brienz']

[(np.str_('2024081109'), np.int64(24)),
 (np.str_('2024081112'), np.int64(24)),
 (np.str_('2024081115'), np.int64(18)),
 (np.str_('2024081118'), np.int64(18)),
 (np.str_('2024081121'), np.int64(12)),
 (np.str_('2024081212'), np.int64(0)),
 (np.str_('2024081309'), np.int64(24)),
 (np.str_('2024081315'), np.int64(0)),
 (np.str_('2024081318'), np.int64(0)),
 (np.str_('2024081512'), np.int64(24)),
 (np.str_('2024081515'), np.int64(18)),
 (np.str_('2024081521'), np.int64(18)),
 (np.str_('2024081603'), np.int64(12)),
 (np.str_('2024081606'), np.int64(6)),
 (np.str_('2024081612'), np.int64(0)),
 (np.str_('2024081615'), np.int64(0)),
 (np.str_('2024082906'), np.int64(6)),
 (np.str_('2024083106'), np.int64(6)),
 (np.str_('2024083109'), np.int64(24)),
 (np.str_('2024083112'), np.int64(24)),
 (np.str_('2024083115'), np.int64(24)),
 (np.str_('2024083118'), np.int64(18)),
 (np.str_('2024083121'), np.int64(12))]

In [27]:
august_warn_times['Morges']

[(np.str_('2024081309'), np.int64(24)),
 (np.str_('2024081312'), np.int64(24)),
 (np.str_('2024081315'), np.int64(18)),
 (np.str_('2024081318'), np.int64(18)),
 (np.str_('2024081618'), np.int64(24)),
 (np.str_('2024081800'), np.int64(0)),
 (np.str_('2024081803'), np.int64(0))]

In [16]:
september_warn_times = load('september_warn_times.pkl')

In [19]:
september_warn_times['Buochs']

[(np.str_('2024090109'), np.int64(0)),
 (np.str_('2024090112'), np.int64(0)),
 (np.str_('2024090115'), np.int64(0)),
 (np.str_('2024090506'), np.int64(6)),
 (np.str_('2024090709'), np.int64(24))]

In [21]:
september_warn_times['Sempach']

[(np.str_('2024090100'), np.int64(12)),
 (np.str_('2024090103'), np.int64(6)),
 (np.str_('2024090106'), np.int64(6)),
 (np.str_('2024090109'), np.int64(0)),
 (np.str_('2024090112'), np.int64(0)),
 (np.str_('2024090115'), np.int64(0)),
 (np.str_('2024090506'), np.int64(6)),
 (np.str_('2024090712'), np.int64(24)),
 (np.str_('2024090715'), np.int64(18)),
 (np.str_('2024090718'), np.int64(18)),
 (np.str_('2024090721'), np.int64(12))]

In [17]:
september_warn_times['Bern']

[(np.str_('2024090115'), np.int64(0))]

In [22]:
september_warn_times['Salavaux']

[(np.str_('2024090106'), np.int64(6))]